## facial point 정보를 수집하여 csv로 저장

* image-url로 부터 facial point 수집하는 api

In [25]:
from __future__ import print_function

In [14]:
#import urllib.parse
#import urllib.request
import json
import requests

## face detect api call 
def face_detect(imgurl):
    url = 'http://apius.faceplusplus.com/v2/detection/detect'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'attribute' : 'none',
              'url' : imgurl,
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
  
    response = requests.post(url,params=values,headers=headers)
    response_data = response.text
    return json.loads(response_data.decode('utf-8'))

In [7]:
face_detect('http://historie.kr/img/face/goodvc_01.jpg')

{u'face': [{u'attribute': {},
   u'face_id': u'957320228be4e9ed9e1b4cb6df23e169',
   u'position': {u'center': {u'x': 46.041667, u'y': 27.5},
    u'eye_left': {u'x': 42.326667, u'y': 23.876583},
    u'eye_right': {u'x': 49.644583, u'y': 23.455722},
    u'height': 18.333333,
    u'mouth_left': {u'x': 43.7025, u'y': 32.486944},
    u'mouth_right': {u'x': 49.655625, u'y': 31.842778},
    u'nose': {u'x': 45.549167, u'y': 28.540833},
    u'width': 13.75},
   u'tag': u''}],
 u'img_height': 360,
 u'img_id': u'a9314d97be0305b5599826fec2c58cd1',
 u'img_width': 480,
 u'session_id': u'07eddd8da7d44379a284a60636f2e596',
 u'url': u'http://historie.kr/img/face/goodvc_01.jpg'}

In [15]:
## face landmark api call by face_id
def face_landmark(faceid):
    url = 'http://apius.faceplusplus.com/v2/detection/landmark'
    headers = { 'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/45.0.2454.101 Safari/537.36',
               'Origin':'http://www.faceplusplus.com', 
               'Accept-Language':'ko-KR,ko;q=0.8,en-US;q=0.6,en;q=0.4', 
               'Content-type':'application/x-www-form-urlencoded', 
               'Accept':'*/*', 
               'Referer':'http://www.faceplusplus.com/demo-landmark/', 
               'Connection':'keep-alive'
              }
    values = {'face_id' : faceid,
              'type' : '83p',
              'api_key' : 'e6f2c06ac2b93b1ae6aad80b90d49eb0',
              'api_secret' : '8o_amgigF4dV4EHZ2OOVEejDeXq4qFab'}
    
    response = requests.post(url,params=values,headers=headers)
    response_data = response.text
    return json.loads(response_data.decode('utf-8'))

def  extract_face_landmark( imgurl, face_index=0 ) :
    fd = face_detect(imgurl)
    face_id = fd['face'][face_index]['face_id']
    return face_landmark(face_id)
    

In [9]:
extract_face_landmark('http://historie.kr/img/face/goodvc_01.jpg')

{u'result': [{u'face_id': u'eebdf5c61c7d49a72f63419f479d195b',
   u'landmark': {u'contour_chin': {u'x': 47.067083, u'y': 39.401944},
    u'contour_left1': {u'x': 39.458125, u'y': 24.21875},
    u'contour_left2': {u'x': 39.194583, u'y': 26.481556},
    u'contour_left3': {u'x': 39.434583, u'y': 28.661111},
    u'contour_left4': {u'x': 39.767917, u'y': 30.751111},
    u'contour_left5': {u'x': 40.352083, u'y': 32.863889},
    u'contour_left6': {u'x': 41.308958, u'y': 34.540833},
    u'contour_left7': {u'x': 42.38625, u'y': 36.264167},
    u'contour_left8': {u'x': 43.623333, u'y': 37.763611},
    u'contour_left9': {u'x': 45.142083, u'y': 39.002222},
    u'contour_right1': {u'x': 54.243958, u'y': 22.742722},
    u'contour_right2': {u'x': 54.4525, u'y': 24.960806},
    u'contour_right3': {u'x': 54.390833, u'y': 27.675639},
    u'contour_right4': {u'x': 54.262917, u'y': 30.14},
    u'contour_right5': {u'x': 53.841042, u'y': 32.415833},
    u'contour_right6': {u'x': 53.051875, u'y': 34.535278},

In [10]:
import json
import numpy as np
from scipy.spatial import distance

def pos_distance(l,r):
#    print(l,r)
    return round(distance.euclidean( [l['x'], l['y']], [r['x'], r['y']] ),5)

def pos_center(l,r):
    return {'x':(l['x']+r['x'])/2, 'y':(l['y']+r['y'])/2}

def extract_face_feature(facial_info):
    cal_items = {'face_width'    :['contour_left9', 'contour_right9']
                 ,'face_height'  :['eyebrow_center','contour_chin']
                 ,'mouth_width'  :['mouth_left_corner','mouth_right_corner']  
                 ,'nose_width'   :['nose_left','nose_right']  
                 ,'eye_width'    :['left_eye_left_corner','right_eye_right_corner']  
                 ,'eye_to_eye'   :['left_eye_center','right_eye_center']  
                 ,'eyebrow_width':['left_eyebrow_upper_middle','left_eyebrow_lower_middle']  
                 ,'lip_to_chin'  :['mouth_lower_lip_bottom','contour_chin']  
                 ,'lip_to_nose'  :['nose_tip','mouth_upper_lip_top']}
    
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    ## add center position
    facial_pos['eyebrow_center'] = pos_center( facial_pos['left_eyebrow_upper_middle'], facial_pos['right_eyebrow_upper_middle']) 
    
    ## calculate key point
    for (k,v) in cal_items.items(): ## iteritems() for python2.7
        keypoint[k] = pos_distance(facial_pos[v[0]], facial_pos[v[1]])
        
    return keypoint

def normalize_dict(data, max_value):
    for k in data.keys():
        data[k] = round(data[k]/max_value,5)
    return data

def get_feature_vector( keypoint ):
    ff = extract_face_feature(keypoint)
    ff = normalize_dict(ff, ff['face_height'])
    return ff

In [11]:
import json
def extract_face_feature_raw(facial_info):
    keypoint = {}
    
    ## json string to object
    info = facial_info
    if type(facial_info) == str:
        info = json.loads(facial_info)
    
    facial_pos = info['result'][0]['landmark']
    dx = facial_pos['contour_left1']['x']
    dy = facial_pos['contour_left1']['y']
    ## calculate key point
    for (k,v) in facial_pos.items(): ## iteritems() for python2.7
        keypoint[k+'_x'] = v['x'] - dx
        keypoint[k+'_y'] = v['y'] - dy
        
    return keypoint

keypoint = extract_face_landmark('http://historie.kr/img/face/goodvc_01.jpg')
extract_face_feature_raw(keypoint)

{u'contour_chin_x': 7.608957999999994,
 u'contour_chin_y': 15.183194,
 u'contour_left1_x': 0.0,
 u'contour_left1_y': 0.0,
 u'contour_left2_x': -0.26354200000000105,
 u'contour_left2_y': 2.262806000000001,
 u'contour_left3_x': -0.023541999999999064,
 u'contour_left3_y': 4.442360999999998,
 u'contour_left4_x': 0.3097919999999945,
 u'contour_left4_y': 6.532361000000002,
 u'contour_left5_x': 0.8939579999999978,
 u'contour_left5_y': 8.645139,
 u'contour_left6_x': 1.8508329999999944,
 u'contour_left6_y': 10.322083,
 u'contour_left7_x': 2.9281249999999943,
 u'contour_left7_y': 12.045417,
 u'contour_left8_x': 4.165208,
 u'contour_left8_y': 13.544860999999997,
 u'contour_left9_x': 5.683957999999997,
 u'contour_left9_y': 14.783472000000003,
 u'contour_right1_x': 14.785832999999997,
 u'contour_right1_y': -1.4760279999999995,
 u'contour_right2_x': 14.994374999999998,
 u'contour_right2_y': 0.7420560000000016,
 u'contour_right3_x': 14.932707999999998,
 u'contour_right3_y': 3.4568890000000003,
 u'con

In [35]:
## (참고용) ls -al | awk '{ print "http://historie.kr/img/face/"$9}' 
## 수집할 image url 
image_data = { 
                'kyumin:1':'http://historie.kr/img/face_x/goodvc_01.jpg'
                ,'kyumin:2':'http://historie.kr/img/face_x/goodvc_02.jpg'
                ,'kyumin:3':'http://historie.kr/img/face_x/goodvc_03.jpg'
 
                ,'GyuseungKyeong:1':'http://historie.kr/img/face/GyuseungKyeong_1.JPG'
                ,'GyuseungKyeong:2':'http://historie.kr/img/face/GyuseungKyeong_2.JPG'
                ,'GyuseungKyeong:3':'http://historie.kr/img/face/GyuseungKyeong_3.JPG'
                ,'daesung:1':'http://historie.kr/img/face/daesung_1.jpg'
                ,'daesung:2':'http://historie.kr/img/face/daesung_2.jpeg'
                ,'daesung:3':'http://historie.kr/img/face/daesung_3.jpg'
                ,'eunjeeshin:1':'http://historie.kr/img/face/eunjeeshin_01.jpg'
                ,'eunjeeshin:2':'http://historie.kr/img/face/eunjeeshin_02.jpg'
                ,'eunjeeshin:3':'http://historie.kr/img/face/eunjeeshin_03.jpg'
                ,'gayeong:1':'http://historie.kr/img/face/gayeong_01.jpg'
                ,'jppark:1':'http://historie.kr/img/face/jppark_02.jpeg'
                ,'jppark:2':'http://historie.kr/img/face/jypark_01.png'
                ,'jppark:3':'http://historie.kr/img/face/jypark_03.jpg'
                ,'kimtaeyoun:1':'http://historie.kr/img/face/kimtaeyoun_01.jpg'
                ,'kimtaeyoun:2':'http://historie.kr/img/face/kimtaeyoun_02.jpg'
                ,'kimtaeyoun:3':'http://historie.kr/img/face/kimtaeyoun_03.png'
                ,'namyoung:1':'http://historie.kr/img/face/namyoung_01.jpg'
                ,'namyoung:2':'http://historie.kr/img/face/namyoung_02.jpg'
                ,'namyoung:3':'http://historie.kr/img/face/namyoung_03.jpg'
                , 'wonbin:1' : 'http://image.fmkorea.com/files/attach/new/20141213/486616/85599953/86802006/da08e25010e6001bf565cd5cca3ca359.jpg'
                , 'wonbin:2' : 'http://cfile25.uf.tistory.com/image/1907CA414F11DCF31C96F2'
                , 'wonbin:3' : 'http://cfile7.uf.tistory.com/image/145E12504DAF7CEE198138'
                , 'kimsoohyun:1' : 'http://cfile10.uf.tistory.com/image/167718454EB74A8C07D3DA'
                , 'kimsoohyun:2' : 'http://image.ytn.co.kr/general/jpg/2013/1218/201312181042081641_h.jpg'
                , 'kimsoohyun:3' : 'http://news.hankyung.com/nas_photo/201205/2012050296441_2012050290011.jpg'
              }

In [36]:
image_data

{'GyuseungKyeong:1': 'http://historie.kr/img/face/GyuseungKyeong_1.JPG',
 'GyuseungKyeong:2': 'http://historie.kr/img/face/GyuseungKyeong_2.JPG',
 'GyuseungKyeong:3': 'http://historie.kr/img/face/GyuseungKyeong_3.JPG',
 'daesung:1': 'http://historie.kr/img/face/daesung_1.jpg',
 'daesung:2': 'http://historie.kr/img/face/daesung_2.jpeg',
 'daesung:3': 'http://historie.kr/img/face/daesung_3.jpg',
 'eunjeeshin:1': 'http://historie.kr/img/face/eunjeeshin_01.jpg',
 'eunjeeshin:2': 'http://historie.kr/img/face/eunjeeshin_02.jpg',
 'eunjeeshin:3': 'http://historie.kr/img/face/eunjeeshin_03.jpg',
 'gayeong:1': 'http://historie.kr/img/face/gayeong_01.jpg',
 'jppark:1': 'http://historie.kr/img/face/jppark_02.jpeg',
 'jppark:2': 'http://historie.kr/img/face/jypark_01.png',
 'jppark:3': 'http://historie.kr/img/face/jypark_03.jpg',
 'kimsoohyun:1': 'http://cfile10.uf.tistory.com/image/167718454EB74A8C07D3DA',
 'kimsoohyun:2': 'http://image.ytn.co.kr/general/jpg/2013/1218/201312181042081641_h.jpg',
 

In [37]:
landmark_dict = {}
for name, img_url in image_data.items(): 
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict[name] = get_feature_vector(keypoint)
    except:
        print ('extract fail : {0}'.format(name))

extract fail : kyumin:1
extract fail : kyumin:3
extract fail : kyumin:2
extract fail : jppark:1


In [38]:
landmark_dict

{'GyuseungKyeong:1': {'eye_to_eye': 0.60659,
  'eye_width': 0.84433,
  'eyebrow_width': 0.05844,
  'face_height': 1.0,
  'face_width': 0.31497,
  'lip_to_chin': 0.17473,
  'lip_to_nose': 0.17407,
  'mouth_width': 0.43512,
  'nose_width': 0.37294},
 'GyuseungKyeong:2': {'eye_to_eye': 0.67471,
  'eye_width': 0.95712,
  'eyebrow_width': 0.06168,
  'face_height': 1.0,
  'face_width': 0.3687,
  'lip_to_chin': 0.19845,
  'lip_to_nose': 0.16256,
  'mouth_width': 0.53248,
  'nose_width': 0.44941},
 'GyuseungKyeong:3': {'eye_to_eye': 0.5475,
  'eye_width': 0.77295,
  'eyebrow_width': 0.05422,
  'face_height': 1.0,
  'face_width': 0.28634,
  'lip_to_chin': 0.20055,
  'lip_to_nose': 0.18315,
  'mouth_width': 0.45304,
  'nose_width': 0.36095},
 'daesung:1': {'eye_to_eye': 0.45481,
  'eye_width': 0.65348,
  'eyebrow_width': 0.06647,
  'face_height': 1.0,
  'face_width': 0.25962,
  'lip_to_chin': 0.23623,
  'lip_to_nose': 0.16433,
  'mouth_width': 0.38874,
  'nose_width': 0.26249},
 'daesung:2': {'e

In [39]:
import pandas as pd
## dict to dataframe
landmark_ds = pd.DataFrame.from_dict( landmark_dict, orient='index')

In [40]:
landmark_ds.head()

,eye_to_eye,eye_width,lip_to_chin,face_width,eyebrow_width,face_height,nose_width,mouth_width,lip_to_nose
GyuseungKyeong:1,0.60659,0.84433,0.17473,0.31497,0.05844,1,0.37294,0.43512,0.17407
GyuseungKyeong:2,0.67471,0.95712,0.19845,0.36870,0.06168,1,0.44941,0.53248,0.16256
GyuseungKyeong:3,0.54750,0.77295,0.20055,0.28634,0.05422,1,0.36095,0.45304,0.18315
daesung:1,0.45481,0.65348,0.23623,0.25962,0.06647,1,0.26249,0.38874,0.16433
daesung:2,0.62668,0.90932,0.23493,0.36580,0.06625,1,0.38984,0.51079,0.17026


# raw facial landmark

In [41]:
!pwd

/Users/goodvc/Data/fc-recsys-school/ch2


In [42]:
## write to csv file
landmark_ds.to_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark.csv')

In [43]:
## load from csv file
landmark_ds = pd.DataFrame.from_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark.csv')

In [44]:
landmark_dict_raw = {}
for name, img_url in image_data.items():
    print (name, ' ' ,img_url)
    try:
        keypoint = extract_face_landmark(img_url)
        landmark_dict_raw[name] = extract_face_feature_raw(keypoint)
    except:
        print ('extract fail : {0}'.format(name))

namyoung:1   http://historie.kr/img/face/namyoung_01.jpg
namyoung:2   http://historie.kr/img/face/namyoung_02.jpg
namyoung:3   http://historie.kr/img/face/namyoung_03.jpg
daesung:1   http://historie.kr/img/face/daesung_1.jpg
daesung:2   http://historie.kr/img/face/daesung_2.jpeg
daesung:3   http://historie.kr/img/face/daesung_3.jpg
kimtaeyoun:2   http://historie.kr/img/face/kimtaeyoun_02.jpg
kimtaeyoun:3   http://historie.kr/img/face/kimtaeyoun_03.png
kimtaeyoun:1   http://historie.kr/img/face/kimtaeyoun_01.jpg
kyumin:1   http://historie.kr/img/face/goodvc_01.jpg
extract fail : kyumin:1
kyumin:3   http://historie.kr/img/face/goodvc_03.jpg
extract fail : kyumin:3
kyumin:2   http://historie.kr/img/face/goodvc_02.jpg
extract fail : kyumin:2
GyuseungKyeong:1   http://historie.kr/img/face/GyuseungKyeong_1.JPG
GyuseungKyeong:2   http://historie.kr/img/face/GyuseungKyeong_2.JPG
GyuseungKyeong:3   http://historie.kr/img/face/GyuseungKyeong_3.JPG
wonbin:1   http://image.fmkorea.com/files/attach

In [51]:
!pwd

/Users/goodvc/Data/fc-recsys-school/ch2


In [52]:
import pandas as pd
## dict to dataframe
landmark_ds = pd.DataFrame.from_dict( landmark_dict, orient='index')
landmark_ds.to_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark.csv')

In [53]:
import pandas as pd
## dict to dataframe
landmark_raw_ds = pd.DataFrame.from_dict( landmark_dict_raw, orient='index')
landmark_raw_ds.to_csv('/Users/goodvc/Data/fc-recsys-school/ch2/resource/landmark_raw.csv')

In [54]:
landmark_raw_ds.head(5).T

,GyuseungKyeong:1,GyuseungKyeong:2,GyuseungKyeong:3,daesung:1,daesung:2
contour_right5_y,16.035834,13.741000,9.305000,17.519667,15.573666
contour_right5_x,32.626888,32.658889,19.332666,40.162833,40.356111
mouth_upper_lip_left_contour3_x,12.810888,14.642445,7.204444,16.832666,19.914778
mouth_upper_lip_left_contour3_y,15.509500,14.335500,8.062166,18.844000,16.001166
left_eye_upper_left_quarter_y,-0.541500,0.218167,-0.386500,-1.888500,-0.909500
left_eye_upper_left_quarter_x,7.384000,9.124000,3.865777,8.313333,10.163222
contour_right7_y,21.664834,17.762500,12.676833,25.733334,21.303500
right_eyebrow_lower_right_quarter_y,-2.844833,-1.564500,-1.747334,-7.955000,-3.494167
left_eye_lower_right_quarter_x,10.930000,12.571333,5.942889,12.442333,14.991445
left_eye_lower_right_quarter_y,1.480000,1.740334,0.456833,-0.074333,0.923500


In [55]:
for name, url in image_data.items():
    print(name, url )

namyoung:1 http://historie.kr/img/face/namyoung_01.jpg
namyoung:2 http://historie.kr/img/face/namyoung_02.jpg
namyoung:3 http://historie.kr/img/face/namyoung_03.jpg
daesung:1 http://historie.kr/img/face/daesung_1.jpg
daesung:2 http://historie.kr/img/face/daesung_2.jpeg
daesung:3 http://historie.kr/img/face/daesung_3.jpg
kimtaeyoun:2 http://historie.kr/img/face/kimtaeyoun_02.jpg
kimtaeyoun:3 http://historie.kr/img/face/kimtaeyoun_03.png
kimtaeyoun:1 http://historie.kr/img/face/kimtaeyoun_01.jpg
kyumin:1 http://historie.kr/img/face/goodvc_01.jpg
kyumin:3 http://historie.kr/img/face/goodvc_03.jpg
kyumin:2 http://historie.kr/img/face/goodvc_02.jpg
GyuseungKyeong:1 http://historie.kr/img/face/GyuseungKyeong_1.JPG
GyuseungKyeong:2 http://historie.kr/img/face/GyuseungKyeong_2.JPG
GyuseungKyeong:3 http://historie.kr/img/face/GyuseungKyeong_3.JPG
wonbin:1 http://image.fmkorea.com/files/attach/new/20141213/486616/85599953/86802006/da08e25010e6001bf565cd5cca3ca359.jpg
wonbin:3 http://cfile7.uf.ti